In [16]:
import os
import sys
sys.path.append('ibmHearingAid/multi_gpu')
import tensorflow as tf
import json

import pitchnet_evaluate_best
import functions_graph_assembly as fga

In [30]:
output_directory = '/saved_models/arch_0628/PND_v04_TLAS_classification0'
config_fn = os.path.join(output_directory, 'config.json')
validation_metrics_fn = os.path.join(output_directory, 'validation_metrics.json')

tfrecords_regex = '/om/user/msaddler/data_pitchnet/bernox2005/FixedFilter_f0min100_f0max300/cf100_species002_spont070/*.tfrecords'

with open(config_fn) as f: CONFIG = json.load(f)

ckpt_num = pitchnet_evaluate_best.get_best_checkpoint_number(validation_metrics_fn,
                                                             metric_key='f0_label:accuracy',
                                                             maximize=True,
                                                             checkpoint_number_key='step')


Selecting checkpoint 54000 (f0_label:accuracy=0.2105589061975479)


In [32]:
ITERATOR_PARAMS = CONFIG['ITERATOR_PARAMS']
batch_size = 128
bytesList_decoding_dict = {"meanrates": {"dtype": "tf.float32", "shape": [100, 500]}}
feature_parsing_dict = pitchnet_evaluate_best.get_feature_parsing_dict_from_tfrecords(tfrecords_regex,
                                                                                      bytesList_decoding_dict)

ITERATOR_PARAMS['feature_parsing_dict'] = feature_parsing_dict
N_CLASSES_DICT = CONFIG['N_CLASSES_DICT']
BRAIN_PARAMS = CONFIG['BRAIN_PARAMS']


tf.reset_default_graph()

# BUILD TFRECORDS ITERATOR GRAPH
iterator, dataset, _ = fga.build_tfrecords_iterator(tfrecords_regex,
                                                    num_epochs=1, shuffle_flag=False,
                                                    batch_size=batch_size,
                                                    iterator_type='one-shot',
                                                    **ITERATOR_PARAMS)
input_tensor_dict = iterator.get_next()

### BUILD BRAIN NETWORK GRAPH
batch_subbands = input_tensor_dict[ITERATOR_PARAMS['feature_signal_path']]
while len(batch_subbands.shape) < 4: batch_subbands = tf.expand_dims(batch_subbands, axis=-1)
batch_out_dict, brain_container = fga.build_brain_graph(batch_subbands, N_CLASSES_DICT, **BRAIN_PARAMS)

### START SESSION AND INITIALIZE GRAPH
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess = tf.Session()
sess.run(init_op)

### BUILD SAVER GRAPH TO LOAD CHECKPOINT
brain_var_scope = 'brain_network'
brain_ckpt_prefix_name = BRAIN_PARAMS.get('save_ckpt_path', 'brain_model.ckpt')
restore_model_path = os.path.join(output_directory, brain_ckpt_prefix_name + '-{}'.format(ckpt_num))
brain_globals = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=brain_var_scope)
brain_locals = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope=brain_var_scope)
brain_variables =  brain_globals + brain_locals
saver_brain_net, out_ckpt_loc_brain_net, brain_net_ckpt = fga.build_saver(
    sess, brain_variables, output_directory,
    restore_model_path=restore_model_path,
    ckpt_prefix_name=brain_ckpt_prefix_name)


delta_f0 dict_keys(['floatList'])
pin_dBSPL dict_keys(['floatList'])
f0_log2 dict_keys(['floatList'])
min_audible_harm dict_keys(['int64List'])
base_f0 dict_keys(['floatList'])
f0_label dict_keys(['int64List'])
f0 dict_keys(['floatList'])
meanrates dict_keys(['bytesList'])
signal dict_keys(['bytesList'])
Ignoring tfrecords_key `signal` (not found in bytesList_decoding_dict)
f0_lognormal dict_keys(['floatList'])
max_audible_harm dict_keys(['int64List'])
low_harm dict_keys(['int64List'])
phase_mode dict_keys(['int64List'])
### Files found: 36
/om/user/msaddler/data_pitchnet/bernox2005/FixedFilter_f0min100_f0max300/cf100_species002_spont070/bez2018meanrates_000000-002016.tfrecords 
...
 /om/user/msaddler/data_pitchnet/bernox2005/FixedFilter_f0min100_f0max300/cf100_species002_spont070/bez2018meanrates_070583-072600.tfrecords
Loading brain network config from /saved_models/arch_0628/PND_v04_TLAS_classification0/brain_arch.json
ADDING OPS TO CHECKPOINTS
[<tf.Tensor 'brain_network/pool_0:0' s

using batch_subbands with final shape: (?, 100, 500, 1)
Loading brain network config from /saved_models/arch_0628/PND_v04_TLAS_classification0/brain_arch.json
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
ADDING OPS TO CHECKPOINTS
[<tf.Tensor 'brain_network_1/pool_0:0' shape=(?, 100, 110, 16) dtype=float32>, <tf.Tensor 'brain_network_1/pool_2:0' shape=(?, 100, 35, 256) dtype=float32>, <tf.Tensor 'brain_network_1/pool_1:0' shape=(?, 100, 46, 32) dtype=float32>, <tf.Tensor 'brain_network_1/pool_5:0' shape=(?, 100, 3, 512) dtype=float32>, <tf.Tensor 'brain_network_1/pool_3:0' shape=(?, 100, 14, 128) d

Selecting checkpoint 54000 (f0_label:accuracy=0.2105589061975479)


In [29]:
brain_container

{'batch_norm_0': <tf.Tensor 'brain_network_1/batch_norm_0/batchnorm/add_1:0' shape=(?, 100, 110, 16) dtype=float32>,
 'batch_norm_1': <tf.Tensor 'brain_network_1/batch_norm_1/batchnorm/add_1:0' shape=(?, 100, 46, 32) dtype=float32>,
 'batch_norm_2': <tf.Tensor 'brain_network_1/batch_norm_2/batchnorm/add_1:0' shape=(?, 100, 35, 256) dtype=float32>,
 'batch_norm_3': <tf.Tensor 'brain_network_1/batch_norm_3/batchnorm/add_1:0' shape=(?, 100, 14, 128) dtype=float32>,
 'batch_norm_4': <tf.Tensor 'brain_network_1/batch_norm_4/batchnorm/add_1:0' shape=(?, 100, 3, 256) dtype=float32>,
 'batch_norm_5': <tf.Tensor 'brain_network_1/batch_norm_5/batchnorm/add_1:0' shape=(?, 100, 3, 512) dtype=float32>,
 'batch_norm_fc_intermediate': <tf.Tensor 'brain_network_1/batch_norm_fc_intermediate/batchnorm/add_1:0' shape=(?, 256) dtype=float32>,
 'conv_0': <tf.Tensor 'brain_network_1/conv_0/BiasAdd:0' shape=(?, 100, 453, 16) dtype=float32>,
 'conv_1': <tf.Tensor 'brain_network_1/conv_1/BiasAdd:0' shape=(?, 1

In [25]:
batch_subbands = input_tensor_dict[ITERATOR_PARAMS['feature_signal_path']]

In [26]:
batch_subbands

<tf.Tensor 'IteratorGetNext:8' shape=(?, 100, 500) dtype=float32>